In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../')

In [3]:
TOTAL_T_STEPS = 144

# Get adjacency matrix for our partitions of Jurbey map

In [4]:
from src.graph_utils import partition_graph_by_lonlat
import networkx as nx
from jurbey.jurbey import JURBEY

with open("../data/1558537930325.jurbey", 'rb') as tempf:
    g = JURBEY.load(tempf.read())
    
g_partition = partition_graph_by_lonlat(g)

A = nx.adjacency_matrix(g_partition)

# Convert timeseries raw data into sparse tensor
Tensor size: 144 timestamps by adjacency matrix of speeds

In [5]:
import pandas

In [6]:
df = pandas.read_csv("../data/timeseries_speed_april_first_week.csv")

In [7]:
df.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,136,137,138,139,140,141,142,143,from_node,to_node
0,result.average,NaN,NaN,NaN,10.854457,NaN,5.680978,7.036838,1.499997,10.088634,...,10.581123,NaN,NaN,NaN,NaN,NaN,10.850843,10.036408,628154368,1023689595
1,result.average,NaN,NaN,NaN,10.854457,NaN,5.680978,7.036838,1.499997,10.088634,...,10.581123,NaN,NaN,NaN,NaN,NaN,10.850843,10.036408,628154368,1023689595
2,result.average,NaN,NaN,NaN,NaN,NaN,10.486210,NaN,NaN,10.937218,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,527147009,27537239
3,result.average,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,527147009,26908815
4,result.average,NaN,NaN,NaN,10.468811,10.636621,10.288534,10.617513,1.758539,3.404401,...,6.098767,10.257657,10.586166,NaN,10.669476,4.313417,10.392901,10.518310,628154370,3804638178


In [8]:
#Our speed data uses segment ids, but the model uses sequential indexes, based on `.nodes()`
import math
id_to_idx = {}
# defaultdict won't do what you expect in Pandas

for id_ in df["from_node"].unique():
    id_to_idx[id_] = math.nan
for id_ in df["to_node"].unique():
    id_to_idx[id_] = math.nan
    
for idx, id_ in enumerate(g_partition.nodes()):
    id_to_idx[id_] = idx

In [9]:
# Let's transform ids to indeces
df["from_node_idx"] = df.replace({"from_node": id_to_idx})["from_node"]
df["to_node_idx"] = df.replace({"to_node": id_to_idx})["to_node"]


In [10]:
df.loc[0][0:-4].values.tolist()

['result.average',
 nan,
 nan,
 nan,
 10.854456901550293,
 nan,
 5.680978015065192,
 7.036837716897327,
 1.499997317790985,
 10.088634490966797,
 10.587458610534668,
 5.728897273540497,
 nan,
 10.424983978271484,
 10.695239067077637,
 10.860746383666992,
 10.227892875671387,
 6.5158751845359815,
 7.626661856969196,
 nan,
 10.888219833374023,
 0.09290917962789537,
 10.04336166381836,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 5.990433931350707,
 10.685218811035156,
 nan,
 nan,
 nan,
 0.12788043916225433,
 10.66620922088623,
 nan,
 10.906200408935547,
 10.888219833374023,
 nan,
 nan,
 nan,
 10.180190086364746,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 0.032882336527109146,
 10.688431739807127,
 nan,
 10.213324069976807,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 10.584891319274902,
 10.4745512008667,
 nan,
 nan,
 10.502264976501463,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 0.03561574593186378,
 nan,
 10.99352741241455,
 0.15259762108325958,
 10.3985667228

In [11]:
from_nodes = df.from_node.values
to_nodes = df.to_node.values
import numpy as np
for edge in g_partition.edges():
    if edge[0] not in from_nodes or edge[1] not in to_nodes:
        a = df.loc[0][0:-4].values
        b = df.loc[0][-4:].values
        df.loc[len(df)] = np.concatenate((a,[edge[0], edge[1]], b[-2:]))


In [12]:
len(df)

6163

## First let's build sparse 3D data tensor

In [13]:
import torch

def snapshot(t, df=df, g_partition=g_partition):
    df_t = df[[t, "from_node_idx", "to_node_idx"]]
    df_t = df_t.dropna()
    row = df_t["from_node_idx"].tolist()
    col = df_t["to_node_idx"].tolist()
    data = df_t[t].tolist()
    size = len(g_partition.nodes()) 

    return {"indices": (row, col), "values": data, "shape": (size, size)}

In [14]:
from scipy.sparse import hstack

def build_sparse_dataset(from_=0, to=TOTAL_T_STEPS):
    dataset = {"indices": ([], [], []), "values": []}
    for t in range(from_, to):

        snap = snapshot(str(t))
        dataset["indices"][0].extend([t] * len(snap["indices"][0]))
        dataset["indices"][1].extend(snap["indices"][0])
        dataset["indices"][2].extend(snap["indices"][1])
        dataset["values"].extend(snap["values"])

    i = torch.LongTensor(dataset["indices"])
    v = torch.FloatTensor(dataset["values"])
    return torch.sparse.FloatTensor(i, v, torch.Size((to, *snap["shape"])))

dataset = build_sparse_dataset()

## Now let's split sparse TxKxK Tensor into 3 TxKxK tensors for training, validation and testing

In [15]:
nonzero_values_cnt = len(dataset._values())
# what percent goes into training/validation/testing
tng_pct = 0.7
val_pct = 0.1
tst_pct = 1 - tng_pct - val_pct
# now we want to split list of all non-zeros promortionally:
# [0, split1_idx], [split1_idx, split2_idx] and [split2_idx:]
split1_idx = int(nonzero_values_cnt * tng_pct)
split2_idx = -int(nonzero_values_cnt * tst_pct)

In [16]:
from random import shuffle

# but we select indexes randomly
idxs = list(range(nonzero_values_cnt))
shuffle(idxs)
# these are non-zero indexes
tng_idxs = idxs[:split1_idx]
val_idxs = idxs[split1_idx:split2_idx]
tst_inxs = idxs[split2_idx:]

In [17]:
144 * 0.7

100.8

In [18]:
dataset_split = {}
for name, idxs in [('tng', tng_idxs), ('val', val_idxs), ('tst', tst_inxs)]:
    i = torch.LongTensor([
        dataset._indices()[0][idxs].tolist(),
        dataset._indices()[1][idxs].tolist(),
        dataset._indices()[2][idxs].tolist()
    ])
    v = torch.FloatTensor(dataset._values()[idxs])
    # NOTE sparse tensor is not supported yet by the model
    dataset_split[name] = torch.sparse.FloatTensor(i, v, dataset.shape).to_dense()



In [22]:
print(len(dataset_split['tng']))

144


# Let's train model

In [20]:
from src.nmf.lsm_rn import LSM_RN
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from test_tube import Experiment
import os

model = LSM_RN(TOTAL_T_STEPS, n=3476, k=50, λ=0.1, adj_mat=A, datasets=dataset_split, batch_size=8)
exp = Experiment(save_dir=os.getcwd())

# pass in experiment for automatic tensorboard logging.    
trainer = Trainer(experiment=exp, max_nb_epochs=30, train_percent_check=1)  
trainer.fit(model)
#TODO lr decay

gpu available: False, used: False
Empty DataFrame
Columns: [Name, Type, Params]
Index: []
G shape: torch.Size([8, 3476, 3476])
t: tensor([52, 79, 96, 80, 83, 26, 20, 16]), batch_nb: 0
G shape: torch.Size([8, 3476, 3476])
t: tensor([ 86, 143,  66,  38,  21,   1,  70,  82]), batch_nb: 1
G shape: torch.Size([8, 3476, 3476])
t: tensor([  5,  47, 101, 141,  10,  12,  88,  81]), batch_nb: 2
G shape: torch.Size([8, 3476, 3476])
t: tensor([ 53,  49,  61,  30, 129,  48,  72, 103]), batch_nb: 3
G shape: torch.Size([8, 3476, 3476])
t: tensor([116,  57,   0, 112,  15,  77,  84,  22]), batch_nb: 4


  3%|▎         | 1/36 [00:00<00:14,  2.41it/s]

G shape: torch.Size([8, 3476, 3476])
t: tensor([16, 27, 64,  7, 66,  5, 88, 23]), batch_nb: 0


  6%|▌         | 2/36 [00:04<00:55,  1.63s/it, batch_nb=0, epoch=0, tng_loss=227.121, v_nb=1]

G shape: torch.Size([8, 3476, 3476])
t: tensor([ 35,  33, 106,  49,  22,   1,  55,  36]), batch_nb: 1


...

G shape: torch.Size([8, 3476, 3476])
t: tensor([121, 108,  71,  48,  17, 105,  72,  39]), batch_nb: 17


100%|██████████| 36/36 [02:13<00:00,  2.78s/it, avg_laplacian_term=69.2, avg_val_fro=32.3, avg_val_loss=101, avg_val_mae=4.52, batch_nb=16, epoch=29, tng_loss=154.016, v_nb=1]

G shape: torch.Size([8, 3476, 3476])
t: tensor([ 68, 112,  18,  52, 139, 108, 116,  12]), batch_nb: 17


100%|██████████| 36/36 [02:16<00:00,  2.78s/it, avg_laplacian_term=69.2, avg_val_fro=32.3, avg_val_loss=101, avg_val_mae=4.52, batch_nb=17, epoch=29, tng_loss=153.924, v_nb=1]

1